In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz
/kaggle/input/shelter-animal-outcomes/train.csv.gz
/kaggle/input/shelter-animal-outcomes/test.csv.gz


In [2]:
# zip 파일 안에 파일 1개일시 압축 안 풀어도 됨
train = pd.read_csv('/kaggle/input/shelter-animal-outcomes/train.csv.gz')
test = pd.read_csv('/kaggle/input/shelter-animal-outcomes/test.csv.gz')

# train test 한꺼번에 출력
display(train.head(), test.head()) ## 정답 column: OutcomeType

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
2,3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
3,4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
4,5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White


In [3]:
train['OutcomeType'].value_counts() # outcome 종류 살펴보기

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: OutcomeType, dtype: int64

In [4]:
# 결측치 확인하기 >> .value_counts(조건 넣어주기)
            #   >> .isnull().sum() #결측치 확인
train.isnull().sum()

AnimalID              0
Name               7691
DateTime              0
OutcomeType           0
OutcomeSubtype    13612
AnimalType            0
SexuponOutcome        1
AgeuponOutcome       18
Breed                 0
Color                 0
dtype: int64

In [5]:
## 전처리 train, test 한꺼번에 >> alldata로 묶어주기

alldata = pd.concat([train, test])
alldata

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,ID
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,NaN
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,NaN
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,NaN
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,NaN
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11451,NaN,NaN,2014-07-08 14:50:00,NaN,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,11452.0
11452,NaN,NaN,2014-10-21 12:57:00,NaN,NaN,Cat,Intact Female,2 weeks,Domestic Shorthair Mix,Blue,11453.0
11453,NaN,NaN,2014-09-29 09:00:00,NaN,NaN,Cat,Intact Female,1 year,Domestic Shorthair Mix,Calico,11454.0
11454,NaN,Rambo,2015-09-05 17:16:00,NaN,NaN,Dog,Neutered Male,6 years,German Shepherd Mix,Black/Tan,11455.0


In [6]:
# AnimalID -> 의미가 있을 수도 있음.. 조심할 것 
# Name 지우는 것? --> but 동물의 이름이 영향 줄 수도 있음
# 이름이 있다? --> 주인이 있다는 뜻도 됨
alldata2 = alldata.drop(columns = ['ID', 'OutcomeType', 'DateTime', 'OutcomeSubtype', 'AnimalID'])
alldata2

,Name,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Hambone,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Emily,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Pearce,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,NaN,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,NaN,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
...,...,...,...,...,...,...
11451,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
11452,NaN,Cat,Intact Female,2 weeks,Domestic Shorthair Mix,Blue
11453,NaN,Cat,Intact Female,1 year,Domestic Shorthair Mix,Calico
11454,Rambo,Dog,Neutered Male,6 years,German Shepherd Mix,Black/Tan


In [7]:
# encoding 진행하기
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
c = alldata2.columns[alldata2.dtypes == object]
for i in c:
    alldata2[i] = le.fit_transform(alldata2[i])
alldata2

,Name,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,2910,1,2,5,1482,146
1,2265,0,3,5,775,184
2,5500,1,2,21,1293,97
3,7968,0,1,26,775,47
4,7968,1,2,21,1101,311
...,...,...,...,...,...,...
11451,7968,0,2,19,775,6
11452,7968,0,0,20,775,46
11453,7968,0,0,5,775,156
11454,5964,1,2,38,841,40


In [8]:
# AgeuponOutcome -> day, week, month, year 기준이 다 다름, 숫자로 인식이 되어야 하는 column
# label encoding시 문제가 생김.
# 연속형 변수로 바꾸어주어야 함

In [9]:
alldata2 = alldata2.fillna(-1) # 결측치 처리
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1)
rf.fit(train2, train['OutcomeType'])
result = rf.predict_proba(test2)
result # 순서 알파벳순서로 들어가 있음

# 분류 대회 -> 확률로 제출하자

array([[0.06      , 0.01      , 0.07      , 0.37      , 0.49      ],
       [0.33      , 0.        , 0.01      , 0.19      , 0.47      ],
       [0.01      , 0.        , 0.        , 0.7       , 0.29      ],
       ...,
       [0.        , 0.        , 0.03344985, 0.        , 0.96655015],
       [0.6       , 0.        , 0.03      , 0.35      , 0.02      ],
       [0.01      , 0.        , 0.21      , 0.23      , 0.55      ]])

In [11]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0
...,...,...,...,...,...,...
11451,11452,1,0,0,0,0
11452,11453,1,0,0,0,0
11453,11454,1,0,0,0,0
11454,11455,1,0,0,0,0


In [12]:
sub.iloc[:,1:] = result
sub

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,0.06000,0.010000,0.07000,0.37000,0.490000
1,2,0.33000,0.000000,0.01000,0.19000,0.470000
2,3,0.01000,0.000000,0.00000,0.70000,0.290000
3,4,0.09000,0.000000,0.07000,0.27000,0.570000
4,5,0.16000,0.000000,0.01000,0.31000,0.520000
...,...,...,...,...,...,...
11451,11452,0.87976,0.000000,0.00000,0.03754,0.082701
11452,11453,0.00000,0.285806,0.00000,0.00000,0.714194
11453,11454,0.00000,0.000000,0.03345,0.00000,0.966550
11454,11455,0.60000,0.000000,0.03000,0.35000,0.020000


In [13]:
sub.to_csv('submission_baseline.csv', index = False)